In [83]:
!which python

/Users/apabook/Desktop/ds-protocol/static/tutorial/venv/bin/python


In [84]:
import json
import requests

# Complete Dataspace Flow


## Consts
Configuration of constant variables used throughout the notebook, including agent URLs and participant IDs.


In [85]:
data_space_provider = "http://127.0.0.1:1200"
data_space_consumer = "http://127.0.0.1:1100"

data_space_provider_participant_id = ""
data_space_provider_participant_token = ""
data_space_provider_dsp_endpoint = ""
data_space_consumer_participant_id = ""
data_space_consumer_participant_token = ""
catalog_id = ""
dataset_id = ""
distribution_id = ""
data_service_id = ""
policy_id = ""
target_id = ""

cn_consumer_id = ""
cn_provider_id = ""
agreement_id = ""
transfer_consumer_id = ""
transfer_provider_id = ""
consumer_data_address = ""

json_header = {
    "Content-Type": "application/json",
}

# Dataspace setup
Authority is created and Consumer Agent and Provider Agent are registered in the Authority. Then, both participants onboard themselves in the system.


## Current Participants Information


In [86]:
url = data_space_provider + "/api/v1/mates/myself"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
data_space_provider_participant_id = response_as_json["participant_id"]
print(json.dumps(response_as_json, indent=4))

{
    "participant_id": "did:jwk:provider",
    "participant_slug": "Myself",
    "participant_type": "Agent",
    "base_url": "http://127.0.0.1:1200",
    "token": null,
    "saved_at": "2026-02-11T11:23:57.816363",
    "last_interaction": "2026-02-11T11:23:57.816419",
    "is_me": true
}


In [87]:
url = data_space_consumer + "/api/v1/mates/myself"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
data_space_consumer_participant_id = response_as_json["participant_id"]
print(json.dumps(response_as_json, indent=4))

{
    "participant_id": "did:jwk:consumer",
    "participant_slug": "Myself",
    "participant_type": "Agent",
    "base_url": "http://127.0.0.1:1100",
    "token": null,
    "saved_at": "2026-02-11T11:24:52.837929",
    "last_interaction": "2026-02-11T11:24:52.837985",
    "is_me": true
}


## Provider and Consumer IDs
Credential exchange and token retrieval for API calls.


In [88]:
url = data_space_provider + "/api/v1/mates/all"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()

for participant in response_as_json:
    if participant.get("participant_type") == "Agent" and participant.get(
            "participant_id") == data_space_consumer_participant_id:
        data_space_consumer_participant_id = participant["participant_id"]
        data_space_consumer_participant_token = participant["token"]
        break

In [89]:
url = data_space_consumer + "/api/v1/mates/all"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()

for participant in response_as_json:
    if participant.get("participant_type") == "Agent" and participant.get(
            "participant_id") == data_space_provider_participant_id:
        data_space_provider_participant_id = participant["participant_id"]
        data_space_provider_participant_token = participant["token"]
        break

In [90]:
print("Provider DID: {}\n".format(data_space_provider_participant_id))
print("Provider token: {}\n".format(data_space_provider_participant_token))
print("Consumer DID: {}\n".format(data_space_consumer_participant_id))
print("Consumer token: {}".format(data_space_consumer_participant_token))

Provider DID: did:jwk:provider

Provider token: b2yNqK0z5_IjrYp09UyL7OOjI1vxKgtgAEHnlf0qns0

Consumer DID: did:jwk:consumer

Consumer token: b2yNqK0z5_IjrYp09UyL7OOjI1vxKgtgAEHnlf0qns0


# Populate catalog

## Create Dataset, Distributions, Policies and Connector Instances
Provider is going to create a catalog, a data service, a dataset and a distribution to be offered to the consumer. The consumer will then discover the offer and accept it.


In [91]:
url = data_space_provider + "/api/v1/catalog-agent/catalogs/main"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
catalog_id = response_as_json["id"]
print(json.dumps(response_as_json, indent=4))

{
    "dctConformsTo": null,
    "dctCreator": null,
    "dctIdentifier": "urn:catalog:074161f3-bfe6-4bbe-a4ad-cec27de5c1ee",
    "dctIssued": "2026-02-11T11:25:15.903619Z",
    "dctModified": null,
    "dctTitle": null,
    "dspaceMainCatalog": true,
    "dspaceParticipantId": "did:jwk:eyJrdHkiOiJSU0EiLCJlIjoiQVFBQiIsIm4iOiIwLW5uempxdDNQVXZtMUVPaEVxNUNxQ05WcEVUM3I4S0Y3VWRKSDJlRFV1dDNhSFlFWGFxbG51SXZ4TUVmN0J1dkhMVnlUaW95U1BoMV9TaDF0ZnBwdF9aNXpXU1ZZUGp0ekZ0QlJsQ0pQYzNfc2YtelRIQURMMmNyYW84WmxXa1FFR2ZtX1g3Y2QtMkd5bmVuR0k0a01rLU9vRERUWEJNRmVoOVdtR1dyMGZBeHJ4Qkp3Unh4QTExODNIOWo1QkY3dlpVZ0Y3akYzV28xcVlva0Z4emJ0Q2FhWWkzTmE3aFYtWWRIYkg3QUhkYnlJa1JvTHhhVG9tb3ktRHRmb0dSSGRzTmpzNDFqZUwxTG5XczJENnVJMm43QTBFQ1hIaHJjZTlMbkt6eXNMWktRM3Ffa2hxR0UzRTZJT1djajRMT1NvNGJ4SXktZzNVT0NpeElSelJ2bFEifQ",
    "foafHomePage": null,
    "id": "urn:catalog:074161f3-bfe6-4bbe-a4ad-cec27de5c1ee"
}


In [92]:
url = data_space_provider + "/api/v1/catalog-agent/data-services/main"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
data_service_id = response_as_json["id"]
print(json.dumps(response_as_json, indent=4))

{
    "catalogId": "urn:catalog:074161f3-bfe6-4bbe-a4ad-cec27de5c1ee",
    "dcatEndpointDescription": null,
    "dcatEndpointUrl": "http://127.0.0.1:1200/dsp/current",
    "dctConformsTo": null,
    "dctCreator": null,
    "dctDescription": null,
    "dctIdentifier": "urn:data-service:86a0a5a1-908e-4317-beca-eea4db8e98db",
    "dctIssued": "2026-02-11T11:25:15.970263Z",
    "dctModified": null,
    "dctTitle": null,
    "dspaceMainDataService": true,
    "id": "urn:data-service:86a0a5a1-908e-4317-beca-eea4db8e98db"
}


In [93]:
url = data_space_provider + "/api/v1/catalog-agent/datasets"
payload = {
    "dctTitle": "My dataspace dataset",
    "catalogId": catalog_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
dataset_id = response_as_json["id"]
print(json.dumps(response_as_json, indent=4))

{
    "catalogId": "urn:catalog:074161f3-bfe6-4bbe-a4ad-cec27de5c1ee",
    "dctConformsTo": null,
    "dctCreator": null,
    "dctDescription": null,
    "dctIdentifier": "urn:dataset:85a7eb56-5048-4a89-84a4-60ccca1a04ed",
    "dctIssued": "2026-02-13T11:52:05.959383Z",
    "dctModified": null,
    "dctTitle": "My dataspace dataset",
    "id": "urn:dataset:85a7eb56-5048-4a89-84a4-60ccca1a04ed"
}


In [94]:
url = data_space_provider + "/api/v1/catalog-agent/distributions"
payload = {
    "dctTitle": "Distribution of my dataset",
    "dctDescription": "This is the distribution of my dataset",
    "dcatAccessService": data_service_id,
    "datasetId": dataset_id,
    "dctFormats": "http+asd"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
distribution_id = response_as_json["id"]
print(json.dumps(response_as_json, indent=4))

{
    "datasetId": "urn:dataset:85a7eb56-5048-4a89-84a4-60ccca1a04ed",
    "dcatAccessService": "urn:data-service:86a0a5a1-908e-4317-beca-eea4db8e98db",
    "dctDescription": "This is the distribution of my dataset",
    "dctFormat": "http+asd",
    "dctIssued": "2026-02-13T11:52:06.176667Z",
    "dctModified": null,
    "dctTitle": "Distribution of my dataset",
    "id": "urn:distribution:df36f651-4354-4e79-b9d9-cf8c4ecfb921"
}


## Create policies with template


In [95]:
url = data_space_provider + "/api/v1/catalog-agent/policy-templates"
response = requests.request("GET", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

[
    {
        "author": "",
        "content": {
            "permission": [
                {
                    "action": "use",
                    "constraint": [
                        {
                            "leftOperand": "bla",
                            "operator": "eq",
                            "rightOperand": "$bla"
                        }
                    ]
                }
            ]
        },
        "date": "2026-02-11T11:25:15.981207Z",
        "description": "Mi example super policy",
        "id": "policy-1",
        "parameters": {
            "$bla": {
                "dataType": "string",
                "restrictions": {},
                "ui": {
                    "hidden": false
                }
            }
        },
        "title": "Mi example policy",
        "version": "1.0"
    }
]


In [96]:
url = data_space_provider + "/api/v1/catalog-agent/policy-templates/instantiate-odrl-offer"
payload = {
    "id": "policy-1",
    "version": "1.0",
    "parameters": {
        "$bla": "did:web:hola.es"
    },
    "entityId": dataset_id,
    "entityType": "Dataset"
}

response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "createdAt": "2026-02-13T11:52:06.664856Z",
    "entity": "urn:dataset:85a7eb56-5048-4a89-84a4-60ccca1a04ed",
    "entityType": "Dataset",
    "id": "urn:odrl-policy:c4dab1b7-2c19-4a96-b086-cb6078f9bc62",
    "instantiationParameters": {
        "$bla": "did:web:hola.es"
    },
    "odrlOffer": {
        "permission": [
            {
                "action": "use",
                "constraint": [
                    {
                        "leftOperand": "bla",
                        "operator": "eq",
                        "rightOperand": "did:web:hola.es"
                    }
                ]
            }
        ]
    },
    "sourceTemplateId": "policy-1",
    "sourceTemplateVersion": "1.0"
}


## Or... just create policies with API


In [97]:
url = data_space_provider + "/api/v1/catalog-agent/odrl-policies"
payload = {
    "odrlOffer": {
        "permission": [
            {
                "action": "use",
                "constraint": [
                    {
                        "rightOperand": "user",
                        "leftOperand": "did:web:hola.es",
                        "operator": "eq"
                    }
                ]
            }

        ],
        "obligation": [],
        "prohibition": []
    },
    "entityId": dataset_id,
    "entityType": "Dataset"
}

response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "createdAt": "2026-02-13T11:52:09.827681Z",
    "entity": "urn:dataset:85a7eb56-5048-4a89-84a4-60ccca1a04ed",
    "entityType": "Dataset",
    "id": "urn:odrl-policy:081a4fa7-b5ba-4a3b-977e-f7ddb25d0211",
    "instantiationParameters": null,
    "odrlOffer": {
        "obligation": [],
        "permission": [
            {
                "action": "use",
                "constraint": [
                    {
                        "leftOperand": "did:web:hola.es",
                        "operator": "eq",
                        "rightOperand": "user"
                    }
                ]
            }
        ],
        "prohibition": []
    },
    "sourceTemplateId": null,
    "sourceTemplateVersion": null
}


## Create and configure connector to final system



In [98]:
url = data_space_provider + "/api/v1/connector/templates"
payload = {
    "authentication": {
        "type": "NO_AUTH"
    },
    "interaction": {
        "mode": "PULL",
        "dataAccess": {
            "protocol": "HTTP",
            "urlTemplate": "{{__ACCESS_URL__}}",
            "method": "{{__ACCESS_METHODS__}}",
            "headers": "{{__ACCESS_HEADERS__}}"
        }
    },
    "parameters": [
        {
            "paramType": "STRING",
            "name": "ACCESS_URL",
            "title": "Url",
            "required": True
        },
        {
            "paramType": "VEC<STRING>",
            "name": "ACCESS_METHODS",
            "title": "Methods",
            "required": True
        },
        {
            "paramType": "MAP<STRING,STRING>",
            "name": "ACCESS_HEADERS",
            "title": "Methods",
            "required": True
        }
    ]
}

response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
connector_name = response_as_json["name"]
connector_version = response_as_json["version"]
print(json.dumps(response_as_json, indent=4))

{
    "name": "urn:connector-template:004ce9ad-2068-4075-a16b-f9b191f86e47",
    "author": "admin",
    "description": null,
    "version": "1.0",
    "createdAt": "2026-02-13T11:52:11.637993Z",
    "authentication": {
        "type": "NO_AUTH"
    },
    "interaction": {
        "mode": "PULL",
        "dataAccess": {
            "protocol": "HTTP",
            "urlTemplate": "{{__ACCESS_URL__}}",
            "method": "{{__ACCESS_METHODS__}}",
            "headers": "{{__ACCESS_HEADERS__}}",
            "bodyTemplate": null
        }
    },
    "parameters": [
        {
            "name": "ACCESS_URL",
            "title": "Url",
            "description": null,
            "paramType": "STRING",
            "required": true,
            "defaultValue": null,
            "autoFilled": false,
            "autoFilledType": null
        },
        {
            "name": "ACCESS_METHODS",
            "title": "Methods",
            "description": null,
            "paramType": "VEC<STRIN

In [99]:
url = data_space_provider + "/api/v1/connector/instances"
payload = {
    "templateName": connector_name,
    "templateVersion": connector_version,
    "distributionId": distribution_id,
    "metadata": {
        "description": "asasdasddas",
        "ownerId": "asdas"
    },
    "dryRun": False,
    "parameters": {
        "ACCESS_URL": "https://dummyjson.com/products",
        "ACCESS_METHODS": ["GET", "HEAD"],
        "ACCESS_HEADERS": {
            "Content-type": "application/json"
        }
    }
}

response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "id": "urn:connector-instance:cfa5e48f-53ca-429a-9393-bd4f334b45d0",
    "name": "urn:connector-template:004ce9ad-2068-4075-a16b-f9b191f86e47",
    "author": null,
    "description": null,
    "version": "1.0",
    "createdAt": "2026-02-13T11:52:11.989567Z",
    "authenticationConfig": {
        "type": "NO_AUTH"
    },
    "interaction": {
        "mode": "PULL",
        "dataAccess": {
            "protocol": "HTTP",
            "urlTemplate": "https://dummyjson.com/products",
            "method": [
                "GET",
                "HEAD"
            ],
            "headers": {
                "Content-type": "application/json"
            },
            "bodyTemplate": null
        }
    },
    "distributionId": "urn:distribution:df36f651-4354-4e79-b9d9-cf8c4ecfb921"
}


## Consumer discovers the provider catalog and finds the offer


In [100]:
url = data_space_consumer + "/dsp/current/catalog/rpc/setup-catalog-request"
payload = {
    "associatedAgentPeer": data_space_provider_participant_id,
    "filter": [], 
    "noCache": True
}

response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
policy_id = response_as_json["response"]["dataset"][-1]["hasPolicy"][0]["@id"]
target_id = response_as_json["response"]["dataset"][0]["@id"]
data_space_provider_dsp_endpoint = response_as_json["response"]["service"]["endpointURL"]
response_as_json


{'request': {'associatedAgentPeer': 'did:jwk:provider',
  'filter': [],
  'noCache': True},
 'response': {'@context': ['https://w3id.org/dspace/2025/1/context.jsonld'],
  '@type': 'Catalog',
  '@id': 'urn:catalog:074161f3-bfe6-4bbe-a4ad-cec27de5c1ee',
  'identifier': 'urn:catalog:074161f3-bfe6-4bbe-a4ad-cec27de5c1ee',
  'issued': '2026-02-11T11:25:15.903619',
  'catalog': [],
  'dataset': [{'@type': 'Dataset',
    '@id': 'urn:dataset:85a7eb56-5048-4a89-84a4-60ccca1a04ed',
    'identifier': 'urn:dataset:85a7eb56-5048-4a89-84a4-60ccca1a04ed',
    'issued': '2026-02-13T11:52:05.959383',
    'title': 'My dataspace dataset',
    'hasPolicy': [{'@id': 'urn:odrl-policy:c4dab1b7-2c19-4a96-b086-cb6078f9bc62',
      'permission': [{'action': 'use',
        'constraint': [{'rightOperand': 'did:web:hola.es',
          'leftOperand': 'bla',
          'operator': 'eq'}]}],
      '@type': 'Offer'},
     {'@id': 'urn:odrl-policy:081a4fa7-b5ba-4a3b-977e-f7ddb25d0211',
      'permission': [{'action': 'u

In [101]:
print("Selected dataset target: \n{}\n".format(target_id))
print("Selected policy id: \n{}\n".format(policy_id))
print("Provider negotiation URL for DSP 2025: \n{}\n".format(data_space_provider_dsp_endpoint))


Selected dataset target: 
urn:dataset:85a7eb56-5048-4a89-84a4-60ccca1a04ed

Selected policy id: 
urn:odrl-policy:c4dab1b7-2c19-4a96-b086-cb6078f9bc62

Provider negotiation URL for DSP 2025: 
http://127.0.0.1:1200/dsp/current



## Consumer discovers own callback address based on DSP version

In [102]:
url = data_space_consumer + "/.well-known/dspace-version/2025-1"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
data_space_consumer_dsp_endpoint = data_space_consumer + response_as_json["path"]
print("Consumer callback address: \n{}\n".format(data_space_consumer_dsp_endpoint))


Consumer callback address: 
http://127.0.0.1:1100/dsp/current



# Contract Negotiation

## Initialization of negotiation request (Consumer -> Provider)

In [156]:
url = data_space_consumer + "/dsp/current/negotiations/rpc/setup-request-init"
payload = {
    "associatedAgentPeer": data_space_provider_participant_id,
    "providerAddress": data_space_provider_dsp_endpoint,
    "callbackAddress": data_space_consumer_dsp_endpoint,
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "use", 
                "constraint": [{
                    "leftOperand": "count", 
                    "operator": "gt", 
                    "rightOperand": "18"
                }]
            }
        ]
    }
}




try:
    response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
    response_as_json = response.json()
    cn_consumer_id = response_as_json["response"]["consumerPid"]
    cn_provider_id = response_as_json["response"]["providerPid"]
    print(json.dumps(response_as_json, indent=2))
except Exception as e:
    print("Error in response, {}.".format(e))

{
  "request": {
    "associatedAgentPeer": "did:jwk:provider",
    "providerAddress": "http://127.0.0.1:1200/dsp/current",
    "callbackAddress": "http://127.0.0.1:1100/dsp/current",
    "offer": {
      "@id": "urn:odrl-policy:c4dab1b7-2c19-4a96-b086-cb6078f9bc62",
      "permission": [
        {
          "action": "use",
          "constraint": [
            {
              "rightOperand": "18",
              "leftOperand": "count",
              "operator": "gt"
            }
          ]
        }
      ],
      "@type": "Offer",
      "target": "urn:dataset:85a7eb56-5048-4a89-84a4-60ccca1a04ed"
    }
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:71173549-dc83-4fb3-a275-446c05bb3dd0",
    "providerPid": "urn:provider-pid:9a8de663-4404-452b-9eca-9f24040c5ac2",
    "state": "REQUESTED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:0e8c

## Provider creates initial offer (Provider -> Consumer)

In [157]:
# Provider creates initial offer (Provider -> Consumer)
url = data_space_provider + "/dsp/current/negotiations/rpc/setup-offer"
payload = {
    "consumerPid": cn_consumer_id,  # remove to test offer from provider
    "providerPid": cn_provider_id,  # remove to test offer from provider
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "use", 
                "constraint": [{
                    "leftOperand": "count", 
                    "operator": "gt", 
                    "rightOperand": "21"
                }]
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

{
  "request": {
    "offer": {
      "@id": "urn:odrl-policy:c4dab1b7-2c19-4a96-b086-cb6078f9bc62",
      "permission": [
        {
          "action": "use",
          "constraint": [
            {
              "rightOperand": "21",
              "leftOperand": "count",
              "operator": "gt"
            }
          ]
        }
      ],
      "@type": "Offer",
      "target": "urn:dataset:85a7eb56-5048-4a89-84a4-60ccca1a04ed"
    },
    "providerPid": "urn:provider-pid:9a8de663-4404-452b-9eca-9f24040c5ac2",
    "consumerPid": "urn:consumer-pid:71173549-dc83-4fb3-a275-446c05bb3dd0"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:71173549-dc83-4fb3-a275-446c05bb3dd0",
    "providerPid": "urn:provider-pid:9a8de663-4404-452b-9eca-9f24040c5ac2",
    "state": "OFFERED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:37d7be99-96c1-46c7-bd

## Consumer sends negotiation request based on offer (Consumer -> Provider)

In [154]:
url = data_space_consumer + "/dsp/current/negotiations/rpc/setup-request"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "use"

            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

{
  "request": {
    "offer": {
      "@id": "urn:odrl-policy:c4dab1b7-2c19-4a96-b086-cb6078f9bc62",
      "permission": [
        {
          "action": "use"
        }
      ],
      "@type": "Offer",
      "target": "urn:dataset:85a7eb56-5048-4a89-84a4-60ccca1a04ed"
    },
    "providerPid": "urn:provider-pid:65885efd-05d8-42e0-9fbb-173b4cb94735",
    "consumerPid": "urn:consumer-pid:671e771e-f209-437e-86de-9a6dc70bc0c6"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:671e771e-f209-437e-86de-9a6dc70bc0c6",
    "providerPid": "urn:provider-pid:65885efd-05d8-42e0-9fbb-173b4cb94735",
    "state": "REQUESTED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:6301e944-717e-441a-832a-37267cd23520",
    "state": "REQUESTED",
    "stateAttribute": null,
    "associatedAgentPeer": "did:jwk:provider",
    "protocol": "DSP",
    "callbackAddress": "http

## Provider updates/confirms the offer (Provider -> Consumer)

In [155]:
url = data_space_provider + "/dsp/current/negotiations/rpc/setup-offer"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "supermegause"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

{
  "request": {
    "offer": {
      "@id": "urn:odrl-policy:c4dab1b7-2c19-4a96-b086-cb6078f9bc62",
      "permission": [
        {
          "action": "supermegause"
        }
      ],
      "@type": "Offer",
      "target": "urn:dataset:85a7eb56-5048-4a89-84a4-60ccca1a04ed"
    },
    "providerPid": "urn:provider-pid:65885efd-05d8-42e0-9fbb-173b4cb94735",
    "consumerPid": "urn:consumer-pid:671e771e-f209-437e-86de-9a6dc70bc0c6"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:671e771e-f209-437e-86de-9a6dc70bc0c6",
    "providerPid": "urn:provider-pid:65885efd-05d8-42e0-9fbb-173b4cb94735",
    "state": "OFFERED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:eec610e2-0d1c-40f1-a4cc-7678a55515a9",
    "state": "OFFERED",
    "stateAttribute": null,
    "associatedAgentPeer": "did:jwk:consumer",
    "protocol": "DSP",
    "callbackAddress": 

## Consumer accepts the offer (Consumer -> Provider)

In [139]:
url = data_space_consumer + "/dsp/current/negotiations/rpc/setup-acceptance"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

{
  "request": {
    "providerPid": "urn:provider-pid:023d2cfb-46c8-4168-b96c-949b45d6a9ee",
    "consumerPid": "urn:consumer-pid:81a385a2-3eb3-42d4-aab5-c1c510ef4ff3"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:81a385a2-3eb3-42d4-aab5-c1c510ef4ff3",
    "providerPid": "urn:provider-pid:023d2cfb-46c8-4168-b96c-949b45d6a9ee",
    "state": "ACCEPTED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:4bd40079-a045-4fba-b122-ae0389bf68cb",
    "state": "ACCEPTED",
    "stateAttribute": null,
    "associatedAgentPeer": "did:jwk:provider",
    "protocol": "DSP",
    "callbackAddress": "http://127.0.0.1:1200/dsp/current",
    "role": "Consumer",
    "properties": {},
    "errorDetails": null,
    "createdAt": "2026-02-13T21:55:05.646807Z",
    "updatedAt": "2026-02-13T21:55:07.071377Z",
    "identifiers": {
      "providerPid": "urn:provider-pid:0

## Provider creates the Agreement (Provider -> Consumer)

In [108]:
url = data_space_provider + "/dsp/current/negotiations/rpc/setup-agreement"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

{
  "request": {
    "providerPid": "urn:provider-pid:c2a3d702-dd56-48d2-9937-117f487affd3",
    "consumerPid": "urn:consumer-pid:e66e62c8-a96c-48ba-ab8f-eeae976919c1"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:e66e62c8-a96c-48ba-ab8f-eeae976919c1",
    "providerPid": "urn:provider-pid:c2a3d702-dd56-48d2-9937-117f487affd3",
    "state": "AGREED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:325453fb-6c11-4160-b6e7-1e0be2f2cd1f",
    "state": "AGREED",
    "stateAttribute": null,
    "associatedAgentPeer": "did:jwk:consumer",
    "protocol": "DSP",
    "callbackAddress": "http://127.0.0.1:1100/dsp/current",
    "role": "Provider",
    "properties": {},
    "errorDetails": null,
    "createdAt": "2026-02-13T11:52:21.630893Z",
    "updatedAt": "2026-02-13T11:52:23.483056Z",
    "identifiers": {
      "providerPid": "urn:provider-pid:c2a3d

## Consumer verifies the agreement (Consumer -> Provider)

In [109]:
url = data_space_consumer + "/dsp/current/negotiations/rpc/setup-verification"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

{
  "request": {
    "providerPid": "urn:provider-pid:c2a3d702-dd56-48d2-9937-117f487affd3",
    "consumerPid": "urn:consumer-pid:e66e62c8-a96c-48ba-ab8f-eeae976919c1"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:e66e62c8-a96c-48ba-ab8f-eeae976919c1",
    "providerPid": "urn:provider-pid:c2a3d702-dd56-48d2-9937-117f487affd3",
    "state": "VERIFIED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:0817b203-d631-4866-be74-8403edf741a4",
    "state": "VERIFIED",
    "stateAttribute": null,
    "associatedAgentPeer": "did:jwk:provider",
    "protocol": "DSP",
    "callbackAddress": "http://127.0.0.1:1200/dsp/current",
    "role": "Consumer",
    "properties": {},
    "errorDetails": null,
    "createdAt": "2026-02-13T11:52:21.666541Z",
    "updatedAt": "2026-02-13T11:52:23.832654Z",
    "identifiers": {
      "providerPid": "urn:provider-pid:c

## Provider finalizes the negotiation (Provider -> Consumer)

In [110]:
url = data_space_provider + "/dsp/current/negotiations/rpc/setup-finalization"
payload = {
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
try:
    agreement = response_as_json
    agreement_id = response_as_json["negotiationAgentModel"]["agreement"]["id"]
    print(json.dumps(response_as_json, indent=2))
except KeyError:
    print("Error in response, check the payload and the endpoint.")
    print(json.dumps(response_as_json, indent=2))
    raise

{
  "request": {
    "providerPid": "urn:provider-pid:c2a3d702-dd56-48d2-9937-117f487affd3",
    "consumerPid": "urn:consumer-pid:e66e62c8-a96c-48ba-ab8f-eeae976919c1"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "consumerPid": "urn:consumer-pid:e66e62c8-a96c-48ba-ab8f-eeae976919c1",
    "providerPid": "urn:provider-pid:c2a3d702-dd56-48d2-9937-117f487affd3",
    "state": "FINALIZED"
  },
  "negotiationAgentModel": {
    "id": "urn:negotiation-process:325453fb-6c11-4160-b6e7-1e0be2f2cd1f",
    "state": "FINALIZED",
    "stateAttribute": null,
    "associatedAgentPeer": "did:jwk:consumer",
    "protocol": "DSP",
    "callbackAddress": "http://127.0.0.1:1100/dsp/current",
    "role": "Provider",
    "properties": {},
    "errorDetails": null,
    "createdAt": "2026-02-13T11:52:21.630893Z",
    "updatedAt": "2026-02-13T11:52:24.228454Z",
    "identifiers": {
      "consumerPid": "urn:consumer-pid

## Final agreement

In [111]:
print("Final agreement: \n{}\n".format(json.dumps(agreement["negotiationAgentModel"]["agreement"], indent=2)))
print("Final agreement id: \n{}\n".format(agreement_id))

Final agreement: 
{
  "id": "urn:agreement:7490d533-c38d-4a8f-946e-c053d07da5b8",
  "negotiationAgentProcessId": "urn:negotiation-process:325453fb-6c11-4160-b6e7-1e0be2f2cd1f",
  "negotiationAgentMessageId": "urn:negotiation-message:da9787a1-5d00-4321-b47a-99316cc26b29",
  "consumerParticipantId": "did:jwk:consumer",
  "providerParticipantId": "did:jwk:provider",
  "agreementContent": {
    "@id": "urn:agreement:7490d533-c38d-4a8f-946e-c053d07da5b8",
    "@type": "Agreement",
    "assignee": "did:jwk:consumer",
    "assigner": "did:jwk:provider",
    "permission": [
      {
        "action": "supermegause"
      }
    ],
    "target": "urn:dataset:85a7eb56-5048-4a89-84a4-60ccca1a04ed",
    "timestamp": "1770983543"
  },
  "target": "urn:dataset:85a7eb56-5048-4a89-84a4-60ccca1a04ed",
  "state": "ACTIVE",
  "createdAt": "2026-02-13T11:52:23.498311Z",
  "updatedAt": "2026-02-13T11:52:24.250666Z"
}

Final agreement id: 
urn:agreement:7490d533-c38d-4a8f-946e-c053d07da5b8



# Transfer Negotiation with Agents


## Transfer request setup (Consumer -> Provider)


In [112]:
url = data_space_consumer + "/dsp/current/transfers/rpc/setup-request"
payload = {
    "associatedAgentPeer": data_space_provider_participant_id,
    "providerAddress": data_space_provider_dsp_endpoint,
    "callbackAddress": data_space_consumer_dsp_endpoint,
    "agreementId": agreement_id,
    "format": "http+asd",
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
transfer_process_consumer_pid = response_as_json["response"]["consumerPid"]
transfer_process_provider_pid = response_as_json["response"]["providerPid"]
print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "associatedAgentPeer": "did:jwk:provider",
    "agreementId": "urn:agreement:7490d533-c38d-4a8f-946e-c053d07da5b8",
    "format": "http+asd",
    "dataAddress": null,
    "providerAddress": "http://127.0.0.1:1200/dsp/current",
    "callbackAddress": "http://127.0.0.1:1100/dsp/current"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "consumerPid": "urn:consumer-pid:02d47617-8721-4b20-b94d-4ad51c602807",
    "providerPid": "urn:provider-pid:c5158781-0faf-488e-9383-95363d5695d9",
    "state": "REQUESTED"
  },
  "transferAgentModel": {
    "id": "urn:transfer-process:3a37d532-1e33-4353-a442-24c364e7014d",
    "state": "REQUESTED",
    "stateAttribute": "OnRequest",
    "associatedAgentPeer": "did:jwk:provider",
    "protocol": "DSP",
    "transferDirection": "http+asd",
    "agreementId": "urn:agreement:7490d533-c38d-4a8f-946e-c053d07da5b8",
    "callbackAddress": "http://127.0.0.1:

## Start transfer (Provider -> Consumer)

In [113]:
url = data_space_provider + "/dsp/current/transfers/rpc/setup-start"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
    "dataAddress": {
        "@type": "DataAddress",
        "endpointType": "https://w3id.org/idsa/v4.1/HTTP",
        "endpoint": "http://example.com",
        "endpointProperties": [
            {
                "@type": "EndpointProperty",
                "name": "authorization",
                "value": "TOKEN-ABCDEFG"
            },
            {
                "@type": "EndpointProperty",
                "name": "authType",
                "value": "bearer"
            }
        ]
    },
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()

print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "consumerPid": "urn:consumer-pid:02d47617-8721-4b20-b94d-4ad51c602807",
    "providerPid": "urn:provider-pid:c5158781-0faf-488e-9383-95363d5695d9",
    "dataAddress": {
      "endpointType": "https://w3id.org/idsa/v4.1/HTTP",
      "endpoint": "http://example.com",
      "endpointProperties": [
        {
          "name": "authorization",
          "value": "TOKEN-ABCDEFG"
        },
        {
          "name": "authType",
          "value": "bearer"
        }
      ]
    }
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "consumerPid": "urn:consumer-pid:02d47617-8721-4b20-b94d-4ad51c602807",
    "providerPid": "urn:provider-pid:c5158781-0faf-488e-9383-95363d5695d9",
    "state": "STARTED"
  },
  "transferAgentModel": {
    "id": "urn:transfer-process:baf6e272-1c4a-4b58-90c8-99dd1efffc3f",
    "state": "STARTED",
    "stateAttribute": "OnRequest",
    "associatedAgentPeer": "did:

## Suspend transfer (Consumer -> Provider)

In [114]:
url = data_space_consumer + "/dsp/current/transfers/rpc/setup-suspension"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
    "code": "SUSPEND",
    "reason": ["Suspending for demonstration purposes"]
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "consumerPid": "urn:consumer-pid:02d47617-8721-4b20-b94d-4ad51c602807",
    "providerPid": "urn:provider-pid:c5158781-0faf-488e-9383-95363d5695d9",
    "code": "SUSPEND",
    "reason": [
      "Suspending for demonstration purposes"
    ]
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "consumerPid": "urn:consumer-pid:02d47617-8721-4b20-b94d-4ad51c602807",
    "providerPid": "urn:provider-pid:c5158781-0faf-488e-9383-95363d5695d9",
    "state": "SUSPENDED"
  },
  "transferAgentModel": {
    "id": "urn:transfer-process:3a37d532-1e33-4353-a442-24c364e7014d",
    "state": "SUSPENDED",
    "stateAttribute": "ByConsumer",
    "associatedAgentPeer": "did:jwk:provider",
    "protocol": "DSP",
    "transferDirection": "http+asd",
    "agreementId": "urn:agreement:7490d533-c38d-4a8f-946e-c053d07da5b8",
    "callbackAddress": "http://127.0.0.1:1200/dsp/current",
    "role": "Consumer",
   

## Restart transfer (Consumer -> Provider)

In [115]:
url = data_space_consumer + "/dsp/current/transfers/rpc/setup-start"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()

print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "consumerPid": "urn:consumer-pid:02d47617-8721-4b20-b94d-4ad51c602807",
    "providerPid": "urn:provider-pid:c5158781-0faf-488e-9383-95363d5695d9",
    "dataAddress": null
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "consumerPid": "urn:consumer-pid:02d47617-8721-4b20-b94d-4ad51c602807",
    "providerPid": "urn:provider-pid:c5158781-0faf-488e-9383-95363d5695d9",
    "state": "STARTED"
  },
  "transferAgentModel": {
    "id": "urn:transfer-process:3a37d532-1e33-4353-a442-24c364e7014d",
    "state": "STARTED",
    "stateAttribute": "ByConsumer",
    "associatedAgentPeer": "did:jwk:provider",
    "protocol": "DSP",
    "transferDirection": "http+asd",
    "agreementId": "urn:agreement:7490d533-c38d-4a8f-946e-c053d07da5b8",
    "callbackAddress": "http://127.0.0.1:1200/dsp/current",
    "role": "Consumer",
    "properties": {},
    "errorDetails": null,
    "createdAt": "2026-02-

## Suspension by Provider (Provider -> Consumer)
(Note: Testing provider-initiated suspension)

In [116]:
url = data_space_provider + "/dsp/current/transfers/rpc/setup-suspension"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
    "code": "SUSPEND",
    "reason": ["Suspending for demonstration purposes"]
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "consumerPid": "urn:consumer-pid:02d47617-8721-4b20-b94d-4ad51c602807",
    "providerPid": "urn:provider-pid:c5158781-0faf-488e-9383-95363d5695d9",
    "code": "SUSPEND",
    "reason": [
      "Suspending for demonstration purposes"
    ]
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "consumerPid": "urn:consumer-pid:02d47617-8721-4b20-b94d-4ad51c602807",
    "providerPid": "urn:provider-pid:c5158781-0faf-488e-9383-95363d5695d9",
    "state": "SUSPENDED"
  },
  "transferAgentModel": {
    "id": "urn:transfer-process:baf6e272-1c4a-4b58-90c8-99dd1efffc3f",
    "state": "SUSPENDED",
    "stateAttribute": "ByProvider",
    "associatedAgentPeer": "did:jwk:consumer",
    "protocol": "DSP",
    "transferDirection": "http+asd",
    "agreementId": "urn:agreement:7490d533-c38d-4a8f-946e-c053d07da5b8",
    "callbackAddress": "http://127.0.0.1:1100/dsp/current",
    "role": "Provider",
   

## Failure Test: Attempt start with invalid parameters

In [117]:
url = data_space_consumer + "/dsp/current/transfers/rpc/setup-start"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
    "dataAddress": None
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "consumerPid": "urn:consumer-pid:02d47617-8721-4b20-b94d-4ad51c602807",
    "providerPid": "urn:provider-pid:c5158781-0faf-488e-9383-95363d5695d9",
    "dataAddress": null
  },
  "error": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferError",
    "consumerPid": null,
    "providerPid": null,
    "code": "5000",
    "reason": [
      "HTTP Error 400 Bad Request: {\"@context\":[\"https://w3id.org/dspace/2025/1/context.jsonld\"],\"@type\":\"TransferError\",\"consumerPid\":null,\"providerPid\":null,\"code\":\"6030\",\"reason\":[\"TransferProcessMessageType TransferStartMessage is not allowed here. Current state is SUSPENDED ByProvider\",\"Failed to parse file\"]}"
    ]
  }
}


## Failure Test: Attempt duplicate or invalid suspension

In [118]:
url = data_space_consumer + "/dsp/current/transfers/rpc/setup-suspension"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
    "code": "SUSPEND",
    "reason": ["Suspending for demonstration purposes"]
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "consumerPid": "urn:consumer-pid:02d47617-8721-4b20-b94d-4ad51c602807",
    "providerPid": "urn:provider-pid:c5158781-0faf-488e-9383-95363d5695d9",
    "code": "SUSPEND",
    "reason": [
      "Suspending for demonstration purposes"
    ]
  },
  "error": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferError",
    "consumerPid": null,
    "providerPid": null,
    "code": "6030",
    "reason": [
      "TransferProcessMessageType TransferSuspensionMessage is not allowed here. Current state is SUSPENDED",
      "Failed to parse file"
    ]
  }
}


## Finalize transfer (Provider -> Consumer)

In [119]:
url = data_space_provider + "/dsp/current/transfers/rpc/setup-completion"
payload = {
    "consumerPid": transfer_process_consumer_pid,
    "providerPid": transfer_process_provider_pid,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "request": {
    "consumerPid": "urn:consumer-pid:02d47617-8721-4b20-b94d-4ad51c602807",
    "providerPid": "urn:provider-pid:c5158781-0faf-488e-9383-95363d5695d9"
  },
  "response": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "consumerPid": "urn:consumer-pid:02d47617-8721-4b20-b94d-4ad51c602807",
    "providerPid": "urn:provider-pid:c5158781-0faf-488e-9383-95363d5695d9",
    "state": "COMPLETED"
  },
  "transferAgentModel": {
    "id": "urn:transfer-process:baf6e272-1c4a-4b58-90c8-99dd1efffc3f",
    "state": "COMPLETED",
    "stateAttribute": "ByProvider",
    "associatedAgentPeer": "did:jwk:consumer",
    "protocol": "DSP",
    "transferDirection": "http+asd",
    "agreementId": "urn:agreement:7490d533-c38d-4a8f-946e-c053d07da5b8",
    "callbackAddress": "http://127.0.0.1:1100/dsp/current",
    "role": "Provider",
    "properties": {},
    "errorDetails": null,
    "createdAt": "2026-02-13T11:52:25.409584Z",